In [1]:
import numpy as np
import cv2

In [2]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png')
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [3]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    # ------------TODO---------------
    # compute point cloud from depth image
    # for-loop is not allowed!!
    # ------------TODO --------------
	alpha, beta = K[0, 0], K[1, 1]
	cx, cy = K[0, 2], K[1, 2]
	mask = seg == 255

	# create a meshgrid for pixel coordinates
	h, w = depth.shape
	u, v = np.meshgrid(np.arange(w), np.arange(h))
	z = depth
	x = (u - cx) * z / alpha
	y = (v - cy) * z / beta
    
	pc = np.stack((x, y, z), axis=-1)[mask]

	print('point cloud shape:', pc.shape)	

	return pc

partial_pc = depth2pc(dpt, seg, K)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

point cloud shape: (19375, 3)


In [4]:
# Save to .obj file and visualize in MeshLab, result can be found in ../results/partial_pc_visual_meshlab.png. 
def save_point_cloud_to_obj(pc, filename):
    """
    Save point cloud to an .obj file.
    Args:
        pc: numpy array of shape (N, 3), where N is the number of points.
        filename: str, the output .obj file path.
    """
    with open(filename, "w") as f:
        for point in pc:
            f.write(f"v {point[0]} {point[1]} {point[2]}\n")
    print(f"Point cloud saved to {filename}")


save_point_cloud_to_obj(partial_pc, "../results/partial_pc.obj")

Point cloud saved to ../results/partial_pc.obj


The MeshLab visulization of partial_pc is shown in the figure below. 

<div style="display: flex; justify-content: space-around;">
    <div>
        <img src="../results/partial_pc_meshlab.png" alt="Partial Point Cloud" style="width: 45%; align: center;">
    </div>
</div>


In [5]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

print('partial pc shape:', partial_pc_sampled.shape)
print('full pc shape:', full_pc_sampled.shape)

# -----------TODO---------------
# implement one way chamfer distance
# -----------TODO---------------

# extent of partial point cloud and full point cloud
partial_pc_sampled = partial_pc_sampled[:, None, :]
full_pc_sampled = full_pc_sampled[None, :, :]

distance_norm =  np.linalg.norm(partial_pc_sampled - full_pc_sampled, axis=2)

# fix x in partial_pc_sampled, compute the min distance over full_pc_sampled
one_way_CD = np.sum(np.min(distance_norm, axis=1)) / partial_pc_sampled.shape[0]
print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', one_way_CD[None])	# extand to 1D array to save

partial pc shape: (2048, 3)
full pc shape: (2048, 3)
one way chamfer distance:  0.009810058088517945
